In [1]:
from collections import defaultdict
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from bokeh.plotting import *
from bokeh.palettes import Spectral11
from numpy import pi

In [2]:
# Build list of files to grab target descriptions from
subdirs = glob.glob('/grp/hst/hstlc/hst13902/filesystem/*')
filelist = []
for subdir in subdirs:
    filelist.append(glob.glob(os.path.join(subdir, '*.fits'))[0])

In [3]:
# Build lists of target descriptions
num_files = len(filelist)
target_descriptions = []
for filename in filelist:
    with fits.open(filename, 'readonly') as hdulist:
        target_descriptions.append(hdulist[0].header.get('TARDESCR','') + hdulist[0].header.get('TARDESC2',''))

In [4]:
# Initialize dictionary
target_dict = {
    'SATELLITE' : 0,
    'PLANET' : 0,
    'ASTEROID' : 0,
    'COMET' : 0,
    'IGM' : 0,
    'ISM' : 0,
    'GALAXY' : 0,
    'QSO' : 0,
    'QUASAR' : 0,
    'STAR' : 0,
    'DA' : 0,
    'DB' : 0
}

In [5]:
# Populate the dictionary
for target in target_descriptions:
    for key in target_dict.keys():
        if key in target:
            target_dict[key] += 1

In [6]:
target_dict

{'ASTEROID': 4,
 'COMET': 16,
 'DA': 149,
 'DB': 22,
 'GALAXY': 705,
 'IGM': 24,
 'ISM': 53,
 'PLANET': 103,
 'QSO': 430,
 'QUASAR': 74,
 'SATELLITE': 45,
 'STAR': 745}

In [9]:
# Make pie chart with matplotlib
fig = plt.figure()
ax1 = fig.add_subplot(111)

# Define the colors
color_map = cm.get_cmap('Spectral')
num_of_colors = len(target_dict.keys())
colors = color_map([x/float(num_of_colors) for x in range(num_of_colors)])

# Plot the pie chart
wedges = ax1.pie(target_dict.values(),
                 labels = target_dict.keys(),
                 startangle=90,
                 labeldistance=1.10,
                 colors=colors)
ax1.axis('equal')

# change the edgecolor for each wedge
for wedge in wedges[0]:
    wedge.set_edgecolor('w')

# Save and close
plt.legend()
plt.savefig('target_pie_chart.png')
plt.close()

In [8]:
# Make pie chart with bokeh

# Build wedges
percents = [0.0]
aggregate_percentage = 0.0
for target in target_dict.keys():
    percentage = target_dict[target] / float(sum(target_dict.values()))
    aggregate_percentage += percentage
    percents.append(aggregate_percentage)
starts = [p*2*pi for p in percents[:-1]]
ends = [p*2*pi for p in percents[1:]]

# Define figure
p = figure(x_range=(-1,1), y_range=(-1,1))
p.wedge(x=0, y=0, radius=1, start_angle=starts, end_angle=ends, color=Spectral11)

# Save figure
output_file("target_pie_chart.html")
show(p)